<a href="https://colab.research.google.com/github/Nai-coder/teaching/blob/master/extractive_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIR - Exercise in Google Colab

## Colab Preparation

Open via google drive -> right click: open with Colab

**Get a GPU**

Toolbar -> Runtime -> Change Runtime Type -> GPU

**Mount Google Drive**

* Download data and clone your github repo to your Google Drive folder
* Use Google Drive as connection between Github and Colab (Could also use direct github access, but re-submitting credentials might be annoying)
* Commit to Github locally from the synced drive

**Keep Alive**

When training google colab tends to kick you out, This might help: https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0

**Get Started**

Run the following script to mount google drive and install needed python packages. Pytorch comes pre-installed.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Breakdown of tasks for part 3:**
1. Run a pre-trained question-answering model on the tuples file
2. use the answers file to evaluate how well the QA worked
3. use top1 documents from part 2, see which of those (queryid, documentid) tuples we have in part 3 and take that subset
4. compare to our file from 1 and the answers file
5. spit out some evaluation charts and add to report

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 66.3 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 83.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip show transformers

Name: transformers
Version: 4.19.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: tqdm, huggingface-hub, regex, numpy, tokenizers, packaging, filelock, requests, pyyaml, importlib-metadata
Required-by: 


In [4]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 65.7 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 140 kB 71.0 MB/s 
     |████████████████████████████████| 212 kB 56.0 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
     |████████████████████████████████| 144 kB 68.1 MB/s 
     |████████████████████████████████| 271 kB 69.5 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 112 kB 77.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
  Attempting unin

In [5]:
num_lines = sum(1 for line in open('/content/drive/MyDrive/Part-3/msmarco-fira-21.qrels.qa-tuples.tsv'))
num_lines

longest_item = print (max(open('/content/drive/MyDrive/Part-3/msmarco-fira-21.qrels.qa-tuples.tsv'), key=len))
longest_item

# max length below produces the length of the line, not number of tabs
file = open("/content/drive/MyDrive/Part-3/msmarco-fira-21.qrels.qa-tuples.tsv")
max_length = 0
for line in file:
    if(len(line) > max_length):
        max_length = len(line)
        max_len_line = line
print(max_len_line)

908316	7892825	3	what to wear to a graduation	What to wear if you’re a graduate: Sensible/comfortable dress shoes. They should look nice and give you the height you want… without giving you blisters or causing you to trip down the steps. Something that won’t look crazy under a gown. If your graduation gown is white, wear white underneath. If you wear a white dress with a yellow pattern under one of the thin white gowns, your gown will look dirty. If you wear any other color underneath, it’ll look like you’ve been in a wet t-shirt grad gown contest. Hair down. If you wear your hair up with a graduation cap, you run the risk of looking bald or having hat troubles. (Also bring a couple of bobby pins to pin to affix your hat to your head.) What to wear if you’re a guest at a graduation: 1  Not jeans. 2  A dress, skirt or dress pants. If it’s a spring/summer graduation, have fun with patterns and colors. December graduations are a good time to embrace textures and jewel tones. 3  Something 

In [6]:
from collections import defaultdict
import pandas as pd

path1 = '/content/drive/MyDrive/Part-3/msmarco-fira-21.qrels.qa-tuples.tsv'
path2 = '/content/drive/MyDrive/Part-3/msmarco-fira-21.qrels.qa-answers.tsv'

#function for loading the data
def read_from_path(path):
  df_dict = defaultdict(list)

  with open(path, "r", encoding="utf") as file:
    for line in file:
      # column_names=['queryid', 'documentid', 'relevance-grade', 'query-text', 'document-text', 'text-selection'])
      splitted = line.split("\t")

      if (len(splitted) >= 5):
        df_dict["queryid"].append(splitted[0])
        df_dict["documentid"].append(splitted[1])
        df_dict["relevance-grade"].append(splitted[2])
        df_dict["query-text"].append(splitted[3])
        df_dict["document-text"].append(splitted[4])
        df_dict["text-selection"].append(splitted[5:])
      else:
        print(f"Wrong number of columns splitted by tab: {len(splitted)}. Required >= 5.")

    return pd.DataFrame.from_dict(df_dict)

def read_qa_from_path(path):
  df_dict = defaultdict(list)

  with open(path, "r", encoding="utf") as file:
    for line in file:
      # column_names=['queryid', 'documentid', 'relevance-grade', 'query-text', 'document-text', 'text-selection'])
      splitted = line.split("\t")

      if (len(splitted) >= 3):
        df_dict["queryid"].append(splitted[0])
        df_dict["documentid"].append(splitted[1])
        df_dict["relevance-grade"].append(splitted[2])
        df_dict["text-selection"].append(" ".join(splitted[3:]))
      else:
        print(f"Wrong number of columns splitted by tab: {len(splitted)}. Required >= 3.")

    return pd.DataFrame.from_dict(df_dict)

df_tuples = read_from_path(path1)
df_answers = read_qa_from_path(path2)

In [7]:
from datasets import load_dataset
#this generic data loading function does not work due to the different number of tabs per line (multiple answers possible per query)

data_path = '/content/drive/MyDrive/Part-3/'
path1 = '/content/drive/MyDrive/Part-3/msmarco-fira-21.qrels.qa-tuples.tsv'
path2 = '/content/drive/MyDrive/Part-3/msmarco-fira-21.qrels.qa-answers.tsv'
path3 = '/content/drive/MyDrive/Part-3/msmarco-fira-21.qrels.retrieval.tsv'

#df_tuples = pd.read_csv(data_path + 'msmarco-fira-21.qrels.qa-tuples.tsv', delimiter='\s')

#df_tuples = load_dataset("csv", data_files= path1, delimiter='\t', column_names=['queryid', 'documentid', 'relevance-grade', 'query-text', 'document-text', 'text-selection'])
#df_answers = load_dataset("csv", data_files= path2, delimiter='\t', column_names=['queryid', 'documentid', 'relevance-grade', 'text-selection'])
#df.retrieval = load_dataset("csv", data_files= path3, column_names=[])

In [8]:
print(df_tuples.head())
df_tuples.shape

   queryid documentid relevance-grade  \
0   135386     100163               3   
1   290779     101026               3   
2    21741    1021598               3   
3   810210    1029662               3   
4  1097448     103635               3   

                                          query-text  \
0                          definition of imagination   
1  how many oscars has clint eastwood won?pdrijgh...   
2         are cold sores and fever blisters the same   
3            what is the cause of blood in the stool   
4              how many calories in slim fast shakes   

                                       document-text  \
0  imagination - the formation of a mental image ...   
1  Clint Eastwood -- five-time Oscar winner and e...   
2  Cold sores, sometimes called fever blisters, a...   
3  Having blood in the stool can be the result of...   
4  The chocolate-flavored shake contains 190, whi...   

                                      text-selection  
0  [, the formation of a

(52606, 6)

In [9]:
print(df_answers.head())
df_answers.shape

   queryid documentid relevance-grade  \
0   135386     100163               3   
1   290779     101026               3   
2    21741    1021598               3   
3   810210    1029662               3   
4  1097448     103635               3   

                                      text-selection  
0   the formation of a mental image of something ...  
1                                             five\n  
2      Cold sores, sometimes called fever blisters\n  
3   wide variety of conditions, such as hemorrhoi...  
4   chocolate-flavored shake contains 190 Cafe Cl...  


(52606, 4)

The loading above is working. Both datasets have the same number of rows. 

Next up we need to load the tuples for processing by a qa trained model.

In [10]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_tuples)
dataset.features #checking loading and column type

{'document-text': Value(dtype='string', id=None),
 'documentid': Value(dtype='string', id=None),
 'query-text': Value(dtype='string', id=None),
 'queryid': Value(dtype='string', id=None),
 'relevance-grade': Value(dtype='string', id=None),
 'text-selection': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [11]:
dataset[1] #checking the second entry in the tuples dataset

{'document-text': "Clint Eastwood -- five-time Oscar winner and eleven-time nominee -- has some advice for people who got snubbed at the Academy Awards this year ... quit whining. Eastwood -- an Academy stalwart -- isn't buying into the protests over the so-called black snub.",
 'documentid': '101026',
 'query-text': 'how many oscars has clint eastwood won?pdrijgheposrgijapeoikgjpesoar',
 'queryid': '290779',
 'relevance-grade': '3',
 'text-selection': ['', 'five\n']}

In [12]:
from transformers import pipeline

qa_model = pipeline("question-answering")
question = "definition of imagination?"
context = "imagination - the formation of a mental image of something that is not perceived as real and is not present to the senses; popular imagination created a world of demons; imagination reveals what the world could be. imaginativeness, vision. creative thinking, creativeness, creativity - the ability to create."
answer = qa_model(question = question, context = context, device = 0)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [13]:
answer

{'answer': 'the formation of a mental image of something that is not perceived as real',
 'end': 88,
 'score': 0.1090235486626625,
 'start': 14}

In [14]:
test = pd.DataFrame.from_dict(answer, orient = 'index')
test

,0
score,0.109024
start,14
end,88
answer,the formation of a mental image of something t...


In [15]:
test.to_csv("answer_save_test.csv", index = False)

In [16]:
model_checkpoint = "huggingface-course/bert-finetuned-squad"
question_answerer = pipeline("question-answering", model = model_checkpoint)
answer2 = question_answerer(question=question, context=context, device = 0)
print(answer2)

Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

{'score': 0.18605996668338776, 'start': 14, 'end': 88, 'answer': 'the formation of a mental image of something that is not perceived as real'}


In [17]:
question = dataset["query-text"]
context = dataset["document-text"]
answers = question_answerer(question = question, context = context, device = 0)

In [18]:
answers

[{'answer': 'the formation of a mental image of something that is not perceived as real',
  'end': 88,
  'score': 0.04405786097049713,
  'start': 14},
 {'answer': 'five-time Oscar winner and eleven',
  'end': 51,
  'score': 0.267705500125885,
  'start': 18},
 {'answer': 'groups of small blisters on the lip and around the mouth',
  'end': 105,
  'score': 0.2090682089328766,
  'start': 49},
 {'answer': 'hemorrhoids',
  'end': 96,
  'score': 0.12018904834985733,
  'start': 85},
 {'answer': '500', 'end': 342, 'score': 0.18758119642734528, 'start': 339},
 {'answer': '8.6 Mbps', 'end': 108, 'score': 0.221059188246727, 'start': 100},
 {'answer': 'varicella-zoster virus',
  'end': 48,
  'score': 0.04803764447569847,
  'start': 26},
 {'answer': 'beneath the surface of the Earth',
  'end': 174,
  'score': 0.4389902651309967,
  'start': 142},
 {'answer': 'sodium, potassium, chloride, and bicarbonate in the body',
  'end': 308,
  'score': 0.0262947715818882,
  'start': 252},
 {'answer': '1,250', '

In [20]:
qa_model = pipeline("question-answering", model = "deepset/roberta-base-squad2")
question = dataset["query-text"]
context = dataset["document-text"]
answers_2 = qa_model(question = question, context = context, device = 0)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [21]:
answers_2

[{'answer': 'the formation of a mental image of something that is not perceived as real',
  'end': 88,
  'score': 0.04405786097049713,
  'start': 14},
 {'answer': 'five-time Oscar winner and eleven',
  'end': 51,
  'score': 0.267705500125885,
  'start': 18},
 {'answer': 'groups of small blisters on the lip and around the mouth',
  'end': 105,
  'score': 0.2090682089328766,
  'start': 49},
 {'answer': 'hemorrhoids',
  'end': 96,
  'score': 0.12018904834985733,
  'start': 85},
 {'answer': '500', 'end': 342, 'score': 0.18758119642734528, 'start': 339},
 {'answer': '8.6 Mbps', 'end': 108, 'score': 0.221059188246727, 'start': 100},
 {'answer': 'varicella-zoster virus',
  'end': 48,
  'score': 0.04803764447569847,
  'start': 26},
 {'answer': 'beneath the surface of the Earth',
  'end': 174,
  'score': 0.4389902651309967,
  'start': 142},
 {'answer': 'sodium, potassium, chloride, and bicarbonate in the body',
  'end': 308,
  'score': 0.0262947715818882,
  'start': 252},
 {'answer': '1,250', '

In [26]:
df_tuples["result_bert_fine"] = answers

In [27]:
df_tuples["result_deepset_roberta"] = answers_2

In [32]:
qa_model_default = pipeline("question-answering")
question = dataset["query-text"]
context = dataset["document-text"]
answers_3 = qa_model_default(question = question, context = context, device = 0)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


In [33]:
df_tuples["result_default"] = answers_3

In [34]:
# save the df_tuples to a df
df_tuples.to_csv("qa-results.csv")

### Evaluation part

In [36]:
df_answers.head(3)

,queryid,documentid,relevance-grade,text-selection
0,135386,100163,3,the formation of a mental image of something ...
1,290779,101026,3,five\n
2,21741,1021598,3,"Cold sores, sometimes called fever blisters\n"


In [38]:
print(answers[0])
print(answers_2[0])
print(answers_3[0])
print(df_answers['text-selection'][0])

{'score': 0.04405786097049713, 'start': 14, 'end': 88, 'answer': 'the formation of a mental image of something that is not perceived as real'}
{'score': 0.05158550664782524, 'start': 215, 'end': 230, 'answer': 'imaginativeness'}
{'score': 0.08691784739494324, 'start': 123, 'end': 168, 'answer': 'popular imagination created a world of demons'}
 the formation of a mental image of something that is not perceived as real and is not present to the senses



In [46]:
df_tuples.head(3)

,queryid,documentid,relevance-grade,query-text,document-text,text-selection,result_bert_fine,result_deepset_roberta,result_default
0,135386,100163,3,definition of imagination,imagination - the formation of a mental image ...,"[, the formation of a mental image of somethin...","{'score': 0.04405786097049713, 'start': 14, 'e...","{'score': 0.05158550664782524, 'start': 215, '...","{'score': 0.08691784739494324, 'start': 123, '..."
1,290779,101026,3,how many oscars has clint eastwood won?pdrijgh...,Clint Eastwood -- five-time Oscar winner and e...,"[, five\n]","{'score': 0.267705500125885, 'start': 18, 'end...","{'score': 0.6737402081489563, 'start': 18, 'en...","{'score': 0.5912112593650818, 'start': 18, 'en..."
2,21741,1021598,3,are cold sores and fever blisters the same,"Cold sores, sometimes called fever blisters, a...","[, Cold sores, sometimes called fever blisters\n]","{'score': 0.2090682089328766, 'start': 49, 'en...","{'score': 0.019526077434420586, 'start': 49, '...","{'score': 0.40041235089302063, 'start': 49, 'e..."


In [52]:
df_tuples['text-selection']

0        [, the formation of a mental image of somethin...
1                                               [, five\n]
2        [, Cold sores, sometimes called fever blisters\n]
3        [, wide variety of conditions, such as hemorrh...
4        [, chocolate-flavored shake contains 190, Cafe...
                               ...                        
52601    [, The twin towers were built in the borough o...
52602    [, To compensate for this your body will requi...
52603    [, Vitamins A, D, and K cooperate synergistica...
52604    [, As soon as your baby can hold his head up s...
52605    [, Call 1-855-236-9236 when you get your new G...
Name: text-selection, Length: 52606, dtype: object

In [51]:
sum(df_tuples['text-selection'] == df_answers['text-selection'][0])

0

In [53]:
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


In [54]:
sum(exact_match_score(df_tuples['text-selection'], df_answers['text-selection'][0]))

AttributeError: ignored

In [43]:
import core_metrics.py as metrics

ModuleNotFoundError: ignored

In [41]:
from datasets import load_metric

metric = load_metric("squad")

In [45]:
theoretical_answers = [
    {ex["queryid"], ex["text-selection"]} for ex in df_answers
]

TypeError: ignored

In [ ]:
theoretical_answers

In [ ]:
print(predicted_answers[0])
print(theoretical_answers[0])

In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)